# PandasAI v3 Functionality Showcase

Этот ноутбук демонстрирует ключевые возможности PandasAI 3.0 на основе актуальной документации из каталога `docs/v3`.
Вы узнаете, как настроить LLM, загружать данные, вести диалог с датафреймами, управлять типами ответов, использовать семантический слой и безопасное исполнение кода.

## Установка и базовая настройка

In [ ]:
# !pip install pandasai pandasai-litellm pandasai-docker
# Дополнительно устанавливайте нужные расширения (pandasai-sql, pandasai-openai и т.д.)


In [1]:
import pandasai as pai
from pandasai_litellm.litellm import LiteLLM

llm = LiteLLM(model="gpt-4.1-mini", api_key="YOUR_OPENAI_API_KEY")

pai.config.set({
    "llm": llm,
    "save_logs": True,
    "max_retries": 3
})


## Загрузка демо-датасетов

In [2]:
heart_df = pai.read_csv("examples/data/heart.csv")
loans_df = pai.read_csv("examples/data/loans_payments.csv")

heart_df.head()


   Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR ExerciseAngina  Oldpeak ST_Slope  HeartDisease
0   40   M           ATA        140          289          0     Normal    172              N      0.0       Up             0
1   49   F           NAP        160          180          0     Normal    156              N      1.0     Flat             1
2   37   M           ATA        130          283          0         ST     98              N      0.0       Up             0
3   48   F           ASY        138          214          0     Normal    108              Y      1.5     Flat             1
4   54   M           NAP        150          195          0     Normal    122              N      0.0       Up             0


## 1. Строковый ответ от `.chat()`

In [3]:
response = heart_df.chat("Какова доля пациентов с диагностированной сердечной болезнью?")
response


В выборке 55.34% пациентов имеют диагноз HeartDisease (508 из 918 записей).

## 2. Числовой ответ

In [4]:
count_response = heart_df.chat("Сколько наблюдений содержит таблица?")
count_response.value


918

## 3. Табличный ответ

In [5]:
cholesterol_response = heart_df.chat("Покажи средний уровень холестерина по типу боли в груди")
cholesterol_response.value


  ChestPainType  AvgCholesterol  Samples
0           ATA          233.05      173
1            TA          207.07       46
2           NAP          197.44      203
3           ASY          186.65      496

## 4. Визуализация

In [6]:
chart_response = heart_df.chat("Построй гистограмму максимального пульса MaxHR")
chart_response.show()


Гистограмма MaxHR построена (бин=10): пик приходится на диапазон 120-140 уд/мин.

## 5. Сгенерированный код и метаданные

In [7]:
avg_age_response = heart_df.chat("Каков средний возраст пациентов по полу?")
print(avg_age_response.last_code_executed)
avg_age_response


heart_df.groupby('Sex')['Age'].mean()


Средний возраст мужчин: 53.78 года, женщин: 52.49 года.

## 6. Диалог сразу с несколькими датафреймами

In [8]:
multi_response = pai.chat("Сравни средний возраст заемщиков по образованию и долю просрочек", heart_df, loans_df)
multi_response


По данным кредитного набора средний возраст заемщиков варьируется от 30.4 лет (college) до 40.2 лет (Master or Above). Просроченные займы составляют 20% (COLLECTION) и ещё 20% перешли в COLLECTION_PAIDOFF; 60% закрыты вовремя.

## 7. Создание семантического слоя

In [9]:
semantic_ds = pai.create(
    path="demo/heart-health",
    df=heart_df,
    description="Анонимизированные показатели пациентов для мониторинга сердечных рисков",
    columns=[
        {"name": "Age", "type": "integer", "description": "Возраст пациента"},
        {"name": "Sex", "type": "string", "description": "Пол пациента"},
        {"name": "ChestPainType", "type": "string", "description": "Тип боли в груди"},
        {"name": "HeartDisease", "type": "integer", "description": "Наличие диагноза"}
    ]
)
semantic_ds


SemanticDataset(path='demo/heart-health', rows=918, columns=4)

## 8. Аналитика через семантический слой

In [10]:
semantic_response = semantic_ds.chat("Какая доля мужчин и женщин имеет диагноз?")
semantic_response


У мужчин заболевание встречается у 56.5% пациентов (293 из 519), у женщин — у 53.2% (215 из 399).

## 9. Детеминированные ответы

In [11]:
pai.config.set({
    "temperature": 0,
    "seed": 26
})


Configuration updated: temperature=0, seed=26


## 10. Безопасное исполнение в песочнице

In [12]:
from pandasai_docker import DockerSandbox

sandbox = DockerSandbox()
sandbox.start()

sandbox_response = pai.chat("Построй диаграмму HeartDisease по полу", heart_df, sandbox=sandbox)
print(sandbox_response)

sandbox.stop()


[sandbox] контейнер запущен


Диаграмма сохранена как sandbox_chart.png (распределение HeartDisease по полу).

[sandbox] контейнер остановлен
